In [1]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [2]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/"

In [3]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [4]:
def file_paths(df):
    # Create different colums for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    mark = ".json"

    df['path_2'] = df['path_2'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_3'] = df['path_3'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_4'] = df['path_4'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [5]:
def string_to_dict(s):
    if ',' not in s and ':' not in s:
        return s
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [6]:
"""
def row_column_paths(df):
# Initialize the new columns
    df['row_path'] = '' 
    df['col__path_1'] = ''  
    df['col__path_2'] = ''  
    df['col__path_3'] = ''
    df['col__path_4'] = ''
    df['col__path_5'] = '' 

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())
        
    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')


    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 
    df['col_path_1_values'] = df.apply(lambda row: [row['value'].get(row['row_path'], None)], axis=1)
    


    # Unlist and store in individual rows
    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    

    # Assign row_path_1_values to row_path_1
    df['col_path_1'] = df['col_path_1_values']
    df = df.explode('col_path_1')


    df['col_path_1_values'] = df['col_path_1_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)

    # For the level 2 keys stored in row_path_1, extract the level 3 keys and store in row_path_2 
    #df['row_path_2_values'] = df.apply(lambda row: [row['row_path_1_values'].get(row['row_path_1'], None)] if isinstance(row['row_path_1_values'], dict) else None, axis=1)
    df['col_path_2_values'] = df.apply(lambda row: [row['col_path_1_values'].get(row['col_path_1'], None)]if isinstance (row['col_path_1_values'], dict) else None, axis=1)
    
    
    # Unlist and store in individual rows
    df['col_path_2_values'] = df['col_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_2_values'] = df['col_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    

    # Assign row_path_2_values to row_path_2
    df['col_path_2'] = df['col_path_2_values']
    df = df.explode('col_path_2')

    df['col_path_2_values'] = df['col_path_2_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    

    #ROW_PATH_3
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['col_path_3_values'] = df.apply(lambda row: [row['col_path_2_values'].get(row['col_path_2'], None)] if isinstance (row['col_path_2_values'], dict) else None, axis=1)

    # Unlist and store in individual rows
    df['col_path_3_values'] = df['col_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_3_values'] = df['col_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    df['col_path_3_values'] = df['col_path_3_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['col_path_3'] = df['col_path_3_values']
    df = df.explode('col_path_3')



    
    #ROW_PATH_4
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['col_path_4_values'] = df.apply(lambda row: [row['col_path_3_values'].get(row['col_path_3'], None)]if isinstance (row['col_path_3_values'], dict) else None, axis=1)
    
    df['col_path_4_values'] = df['col_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_4_values'] = df['col_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df[['col_path_3', 'col_path_4_values']])
    
    df['col_path_4_values'] = df['col_path_4_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['col_path_4'] = df['col_path_4_values']
    df = df.explode('col_path_4')


    #ROW_PATH_5
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['col_path_5_values'] = df.apply(lambda row: [row['col_path_4_values'].get(row['col_path_4'], None)]if isinstance (row['col_path_4_values'], dict) else None, axis=1)
    
    df['col_path_5_values'] = df['col_path_5_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_5_values'] = df['col_path_5_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df[['col_path_4', 'col_path_5_values']])
    
    df['col_path_5_values'] = df['col_path_5_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    # Assign row_path_3_values to row_path_3
    df['col_path_5'] = df['col_path_5_values']
    df = df.explode('col_path_5')
    
   

    return df
"""

"\ndef row_column_paths(df):\n# Initialize the new columns\n    df['row_path'] = '' \n    df['col__path_1'] = ''  \n    df['col__path_2'] = ''  \n    df['col__path_3'] = ''\n    df['col__path_4'] = ''\n    df['col__path_5'] = '' \n\n    # Extract the level 1 keys \n    for index, row in df.iterrows():\n        df.at[index, 'row_path'] = list(row['value'].keys())\n\n    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]\n\n    # Take the level 1 keys stored in a list and store them in individual rows\n    df = df.explode('row_path')\n\n\n    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 \n    df['col_path_1_values'] = df.apply(lambda row: [row['value'].get(row['row_path'], None)], axis=1)\n\n\n\n    # Unlist and store in individual rows\n    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)\n    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstan

In [7]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = '' 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())

    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')

    def column_paths(df, row_var, get_var, unlist_var, explode_var):

        print("before", df[row_var].apply(type).value_counts())
        
        # Convert to dict if it's a string
        df[row_var] = df[row_var].apply(
            lambda x: string_to_dict(x) if isinstance(x, str) else x)
        
        print("After", df[row_var].apply(type).value_counts())
        
        # Extract nested values using keys
        df[unlist_var] = df.apply(lambda row: [row[row_var].get(row[get_var], None)] if isinstance (row[row_var], dict) else None, axis=1)


        # Unlist and explode
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        
        df[explode_var] = df[unlist_var]
        df = df.explode(explode_var)

    

        return df

    row_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['row_path', 'col_path_1', 'col_path_2', 'col_path_3', 'col_path_4']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']

    for r, g, u, e in zip(row_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        row_var = unlist_var

    return df


In [8]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values', 'col_path_5_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}Instagram/Output/Output_" + file_name + '.csv', index=False)

    return df

In [9]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)

    
    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)
    display(df)

    # Delete user specific informations
    df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)
    display(df)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [10]:
input_directory = Path(f'{main_path}Instagram/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input


In [11]:

for file in input_directory.iterdir():  
    if file.is_file():  
        print(file)
        structure_donations(file)
        

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/08-04-2025-json_structure instagram.json


,connections/followers_and_following/follow_requests_you've_received.json,connections/followers_and_following/followers_1.json,connections/followers_and_following/following.json,connections/followers_and_following/removed_suggestions.json,security_and_login_information/login_and_profile_creation/last_known_location.json,security_and_login_information/login_and_profile_creation/login_activity.json,security_and_login_information/login_and_profile_creation/password_change_activity.json,security_and_login_information/login_and_profile_creation/profile_privacy_changes.json,security_and_login_information/login_and_profile_creation/signup_details.json,personal_information/information_about_you/possible_emails.json,...,your_instagram_activity/comments/post_comments_1.json,preferences/settings/use_cross-app_messaging.json,preferences/settings/notification_preferences.json,preferences/settings/comments_allowed_from.json,preferences/settings/consents.json,logged_information/policy_updates_and_permissions/notification_of_privacy_policy_updates.json,apps_and_websites_off_of_instagram/apps_and_websites/your_activity_off_meta_technologies_settings.json,apps_and_websites_off_of_instagram/apps_and_websites/your_activity_off_meta_technologies.json,ads_information/instagram_ads_and_businesses/advertisers_using_your_activity_or_information.json,connections/contacts/synced_contacts.json
0,{'relationships_follow_requests_received': [{'...,"[{'title': 'string', 'media_list_data': 'array...",{'relationships_following': [{'title': 'string...,{'relationships_dismissed_suggested_users': [{...,{'account_history_imprecise_last_known_locatio...,{'account_history_login_history': [{'title': '...,{'account_history_password_change_history': [{...,{'account_history_account_privacy_history': [{...,{'account_history_registration_info': [{'title...,{'inferred_data_inferred_emails': [{'title': '...,...,"{'media_list_data': [{'uri': 'string'}], 'stri...",{'settings_upgraded_to_cross_app_messaging': [...,{'settings_notification_preferences': [{'strin...,{'settings_allow_comments_from': [{'title': 's...,"{'timestamp': 'number', 'media': 'array', 'lab...",{'policy_updates_and_permissions_notification_...,"{'timestamp': 'number', 'media': 'array', 'lab...",{'apps_and_websites_off_meta_activity': [{'nam...,{'ig_custom_audiences_all_types': [{'advertise...,"{'contacts_contact_info': [{'title': 'string',..."


,followers_and_following/follow_requests_you've_received.json,followers_and_following/followers_1.json,followers_and_following/following.json,followers_and_following/removed_suggestions.json,login_and_profile_creation/last_known_location.json,login_and_profile_creation/login_activity.json,login_and_profile_creation/password_change_activity.json,login_and_profile_creation/profile_privacy_changes.json,login_and_profile_creation/signup_details.json,information_about_you/possible_emails.json,...,comments/post_comments_1.json,settings/use_cross-app_messaging.json,settings/notification_preferences.json,settings/comments_allowed_from.json,settings/consents.json,policy_updates_and_permissions/notification_of_privacy_policy_updates.json,apps_and_websites/your_activity_off_meta_technologies_settings.json,apps_and_websites/your_activity_off_meta_technologies.json,instagram_ads_and_businesses/advertisers_using_your_activity_or_information.json,contacts/synced_contacts.json
0,{'relationships_follow_requests_received': [{'...,"[{'title': 'string', 'media_list_data': 'array...",{'relationships_following': [{'title': 'string...,{'relationships_dismissed_suggested_users': [{...,{'account_history_imprecise_last_known_locatio...,{'account_history_login_history': [{'title': '...,{'account_history_password_change_history': [{...,{'account_history_account_privacy_history': [{...,{'account_history_registration_info': [{'title...,{'inferred_data_inferred_emails': [{'title': '...,...,"{'media_list_data': [{'uri': 'string'}], 'stri...",{'settings_upgraded_to_cross_app_messaging': [...,{'settings_notification_preferences': [{'strin...,{'settings_allow_comments_from': [{'title': 's...,"{'timestamp': 'number', 'media': 'array', 'lab...",{'policy_updates_and_permissions_notification_...,"{'timestamp': 'number', 'media': 'array', 'lab...",{'apps_and_websites_off_meta_activity': [{'nam...,{'ig_custom_audiences_all_types': [{'advertise...,"{'contacts_contact_info': [{'title': 'string',..."


before value
<class 'dict'>    35
Name: count, dtype: int64
After value
<class 'dict'>    35
Name: count, dtype: int64
before col_path_1_values
<class 'dict'>    70
<class 'str'>      9
Name: count, dtype: int64
After col_path_1_values
<class 'dict'>    70
<class 'str'>      9
Name: count, dtype: int64
before col_path_2_values
<class 'dict'>        90
<class 'str'>         30
<class 'NoneType'>     9
Name: count, dtype: int64
After col_path_2_values
<class 'dict'>        90
<class 'str'>         30
<class 'NoneType'>     9
Name: count, dtype: int64
before col_path_3_values
<class 'dict'>        160
<class 'NoneType'>     56
<class 'str'>          15
Name: count, dtype: int64
After col_path_3_values
<class 'dict'>        160
<class 'NoneType'>     56
<class 'str'>          15
Name: count, dtype: int64
before col_path_4_values
<class 'str'>         160
<class 'NoneType'>     71
Name: count, dtype: int64
After col_path_4_values
<class 'str'>         160
<class 'NoneType'>     71
Name: cou

,your_instagram_activity/content/profile_photos.json,your_instagram_activity/content/stories.json,your_instagram_activity/messages/inbox/annasalasitjar_1366882294712103/message_1.json,your_instagram_activity/messages/inbox/benmolinejorge_1501590597904727/message_1.json,your_instagram_activity/messages/inbox/carles_1573430487391015/message_1.json,your_instagram_activity/messages/inbox/cbcnol_1437962157594010/message_1.json,your_instagram_activity/messages/inbox/cristinabayesquer_1440752623990169/message_1.json,your_instagram_activity/messages/inbox/holly_1448313899893386/message_1.json,your_instagram_activity/messages/inbox/sarabayesquer_1360017525390160/message_1.json,your_instagram_activity/messages/secret_conversations.json,...,security_and_login_information/login_and_profile_creation/last_known_location.json,security_and_login_information/login_and_profile_creation/instagram_signup_details.json,security_and_login_information/login_and_profile_creation/profile_privacy_changes.json,logged_information/recent_searches/profile_searches.json,your_instagram_activity/messages/inbox/albabayes_1432638578129870/message_1.json,ads_information/ads_and_topics/posts_viewed.json,ads_information/ads_and_topics/videos_watched.json,connections/followers_and_following/following.json,ads_information/instagram_ads_and_businesses/advertisers_using_your_activity_or_information.json,connections/contacts/synced_contacts.json
0,"{'ig_profile_picture': [{'uri': 'string', 'cre...","{'ig_stories': [{'uri': 'string', 'creation_ti...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...",{'ig_secret_conversations': {'armadillo_device...,...,{'account_history_imprecise_last_known_locatio...,{'account_history_registration_info': [{'title...,{'account_history_account_privacy_history': [{...,"{'searches_user': [{'title': 'string', 'media_...","{'participants': [{'name': 'string'}], 'messag...",{'impressions_history_posts_seen': [{'string_m...,{'impressions_history_videos_watched': [{'stri...,{'relationships_following': [{'title': 'string...,{'ig_custom_audiences_all_types': [{'advertise...,"{'contacts_contact_info': [{'title': 'string',..."


...

,activity_messages/events_interactions.json,activity_messages/people_and_friends.json,activity_messages/milestone_notifications.json,in-app_messages/your_in-app_messages_interactions.json,location/device_location.json,location/primary_location.json,location/primary_public_location.json,location/timezone.json,location/your_privacy_jurisdiction.json,other_logged_information/consents.json,...,messages/inbox/markschuckman_1119474951399556/message_1.json,messages/filtered_threads/love_749_8211296205611804/message_1.json,messages/filtered_threads/privatedatingsite956_8792251370847801/message_1.json,messages/inbox/olivierrenecarstenandnils_289857417783733/message_1.json,account_activity.json,messages/inbox/facebookuser_10205079055275834/message_1.json,comments_and_reactions/likes_and_reactions.json,your_activity_off_meta_technologies.json,messages/inbox/henkbierlee_1096317910381927/message_1.json,messages/inbox/1123750567638661/message_1.json
0,"{'media': 'array', 'label_values': [{'label': ...","[{'timestamp': 'number', 'media': 'array', 'la...","[{'timestamp': 'number', 'media': 'array', 'la...","[{'timestamp': 'number', 'media': 'array', 'la...",{'phone_number_location_v2': [{'spn': 'string'...,{'primary_location_v2': {'city_region_pairs': ...,{'primary_public_location_v2': {'city': 'strin...,"{'media': 'array', 'label_values': [{'label': ...","{'media': 'array', 'label_values': [{'label': ...","{'timestamp': 'number', 'media': 'array', 'lab...",...,"{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...","{'account_activity_v2': [{'action': 'string', ...","{'participants': [{'name': 'string'}], 'messag...","[{'timestamp': 'number', 'media': 'array', 'la...","[{'title': 'string', 'media': 'array', 'label_...","{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag..."


before value
<class 'dict'>    374
Name: count, dtype: int64
After value
<class 'dict'>    374
Name: count, dtype: int64
before col_path_1_values
<class 'dict'>    424
<class 'str'>     119
Name: count, dtype: int64
After col_path_1_values
<class 'dict'>    424
<class 'str'>     119
Name: count, dtype: int64
before col_path_2_values
<class 'str'>         387
<class 'NoneType'>    119
<class 'dict'>         98
<class 'list'>          1
Name: count, dtype: int64
After col_path_2_values
<class 'str'>         387
<class 'NoneType'>    119
<class 'dict'>         98
<class 'list'>          1
Name: count, dtype: int64
before col_path_3_values
<class 'NoneType'>    507
<class 'str'>          79
<class 'dict'>         34
Name: count, dtype: int64
After col_path_3_values
<class 'NoneType'>    507
<class 'str'>          79
<class 'dict'>         34
Name: count, dtype: int64
before col_path_4_values
<class 'NoneType'>    586
<class 'str'>          28
<class 'dict'>         12
Name: count, dtype: i

,connections/followers_and_following/close_friends.json,connections/followers_and_following/follow_requests_you've_received.json,connections/followers_and_following/followers_1.json,connections/followers_and_following/following.json,connections/followers_and_following/recent_follow_requests.json,connections/followers_and_following/recently_unfollowed_profiles.json,ads_information/instagram_ads_and_businesses/other_categories_used_to_reach_you.json,ads_information/instagram_ads_and_businesses/information_you've_submitted_to_advertisers.json,ads_information/instagram_ads_and_businesses/subscription_for_no_ads.json,your_instagram_activity/messages/inbox/adriaanvandis_1563433145050304/message_1.json,...,your_instagram_activity/messages/inbox/hesterjansen_1039224930809897/message_1.json,your_instagram_activity/messages/inbox/munekas_6940793355998422/message_1.json,your_instagram_activity/story_sticker_interactions/story_likes.json,your_instagram_activity/content/stories.json,ads_information/ads_and_topics/ads_viewed.json,ads_information/ads_and_topics/posts_viewed.json,apps_and_websites_off_of_instagram/apps_and_websites/your_activity_off_meta_technologies.json,ads_information/ads_and_topics/videos_watched.json,ads_information/instagram_ads_and_businesses/advertisers_using_your_activity_or_information.json,your_instagram_activity/messages/inbox/luuk_571474510914761/message_1.json
0,{'relationships_close_friends': [{'title': 'st...,{'relationships_follow_requests_received': [{'...,"[{'title': 'string', 'media_list_data': 'array...",{'relationships_following': [{'title': 'string...,{'relationships_permanent_follow_requests': [{...,{'relationships_unfollowed_users': [{'title': ...,"{'media': 'array', 'label_values': [{'label': ...","{'ig_lead_gen_info': [{'label': 'string', 'val...","{'media': 'array', 'label_values': [{'label': ...","{'participants': [{'name': 'string'}], 'messag...",...,"{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...",{'story_activities_story_likes': [{'title': 's...,"{'ig_stories': [{'uri': 'string', 'creation_ti...",{'impressions_history_ads_seen': [{'string_map...,{'impressions_history_posts_seen': [{'string_m...,{'apps_and_websites_off_meta_activity': [{'nam...,{'impressions_history_videos_watched': [{'stri...,{'ig_custom_audiences_all_types': [{'advertise...,"{'participants': [{'name': 'string'}], 'messag..."


,followers_and_following/close_friends.json,followers_and_following/follow_requests_you've_received.json,followers_and_following/followers_1.json,followers_and_following/following.json,followers_and_following/recent_follow_requests.json,followers_and_following/recently_unfollowed_profiles.json,instagram_ads_and_businesses/other_categories_used_to_reach_you.json,instagram_ads_and_businesses/information_you've_submitted_to_advertisers.json,instagram_ads_and_businesses/subscription_for_no_ads.json,messages/inbox/adriaanvandis_1563433145050304/message_1.json,...,messages/inbox/hesterjansen_1039224930809897/message_1.json,messages/inbox/munekas_6940793355998422/message_1.json,story_sticker_interactions/story_likes.json,content/stories.json,ads_and_topics/ads_viewed.json,ads_and_topics/posts_viewed.json,apps_and_websites/your_activity_off_meta_technologies.json,ads_and_topics/videos_watched.json,instagram_ads_and_businesses/advertisers_using_your_activity_or_information.json,messages/inbox/luuk_571474510914761/message_1.json
0,{'relationships_close_friends': [{'title': 'st...,{'relationships_follow_requests_received': [{'...,"[{'title': 'string', 'media_list_data': 'array...",{'relationships_following': [{'title': 'string...,{'relationships_permanent_follow_requests': [{...,{'relationships_unfollowed_users': [{'title': ...,"{'media': 'array', 'label_values': [{'label': ...","{'ig_lead_gen_info': [{'label': 'string', 'val...","{'media': 'array', 'label_values': [{'label': ...","{'participants': [{'name': 'string'}], 'messag...",...,"{'participants': [{'name': 'string'}], 'messag...","{'participants': [{'name': 'string'}], 'messag...",{'story_activities_story_likes': [{'title': 's...,"{'ig_stories': [{'uri': 'string', 'creation_ti...",{'impressions_history_ads_seen': [{'string_map...,{'impressions_history_posts_seen': [{'string_m...,{'apps_and_websites_off_meta_activity': [{'nam...,{'impressions_history_videos_watched': [{'stri...,{'ig_custom_audiences_all_types': [{'advertise...,"{'participants': [{'name': 'string'}], 'messag..."


before value
<class 'dict'>    99
Name: count, dtype: int64
After value
<class 'dict'>    99
Name: count, dtype: int64
before col_path_1_values
<class 'dict'>    184
<class 'str'>       5
Name: count, dtype: int64
After col_path_1_values
<class 'dict'>    184
<class 'str'>       5
Name: count, dtype: int64
before col_path_2_values
<class 'str'>         126
<class 'dict'>        125
<class 'NoneType'>      5
Name: count, dtype: int64
After col_path_2_values
<class 'str'>         126
<class 'dict'>        125
<class 'NoneType'>      5
Name: count, dtype: int64
before col_path_3_values
<class 'dict'>        185
<class 'NoneType'>    148
<class 'str'>          35
Name: count, dtype: int64
After col_path_3_values
<class 'dict'>        185
<class 'NoneType'>    148
<class 'str'>          35
Name: count, dtype: int64
before col_path_4_values
<class 'NoneType'>    183
<class 'str'>         182
<class 'dict'>          6
Name: count, dtype: int64
After col_path_4_values
<class 'NoneType'>    183

In [12]:
# Path to the folder containing CSV files
output_path = f"{main_path}Instagram/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values', 'col_path_5_values']


merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest  #
    merged_df = merged_df.merge(df, on=common_columns, how="outer")




# Filter where col1 contains 'messages', then drop duplicates based on col2
df_filtered = merged_df[merged_df["path_1"] == "messages"].drop_duplicates(subset="path_2")


# Append rows where col1 does not contain 'messages'
df_final = pd.concat([df_filtered, merged_df[merged_df["path_1"] != "messages"]], ignore_index=True)
df_final = df_final.replace('Missing', np.nan)


print(df_final.isna().sum())

# Save the final merged DataFrame
df_final.to_csv(f"{main_path}Instagram/Final/Merged_structures_IG.csv", index=False)

variable                0
value                   0
path_1                  0
path_2               1717
path_3               2545
path_4               3212
json_name               0
row_path                0
col_path_1            474
col_path_2           1908
col_path_3           2432
col_path_4           3107
col_path_5           3196
data_type              70
col_path_1_values       0
col_path_2_values     474
col_path_3_values    1908
col_path_4_values    2432
col_path_5_values    3107
dtype: int64


/tmp/ipykernel_102350/1853675807.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.replace('Missing', np.nan)
